In [16]:
import pandas as pd
from datetime import datetime, timedelta
import time
from ipywidgets import HTML
from ipyleaflet import Map, Marker, Popup, basemaps, AwesomeIcon
from IPython.display import display
import mysql.connector
import configparser

#Reading the configurations from the config file.
config = configparser.ConfigParser()
config.read('config.ini')
#Getting the update interval from config
update_interval =int(config.get("app", "update_interval"))
#Connecting to the MySQL database.
db = mysql.connector.connect(
    host=config.get("app", "host"),
    user=config.get("app", "user"),
    password=config.get("app", "password"),
    database=config.get("app", "database")
)
#Creating the cursor method to execute MySQL Queries.
cursor = db.cursor()

data_path = config.get("app", "data_path")
stops = pd.read_csv(f"{data_path}/stops.txt")
routes = pd.read_csv(f"{data_path}/routes.txt")
trips = pd.read_csv(f"{data_path}/trips.txt")
stop_times = pd.read_csv(f"{data_path}/stop_times.txt")
calendar = pd.read_csv(f"{data_path}/calendar.txt")



#Defining the cursors specialties. If the data with the same trip_id already exist in the table, it only executes the update function instead of inserting and updating together to avoid duplicates.
insert_or_update_buses = ("""
INSERT INTO buses 
(route_short_name, trip_id, service_id, route_id, direction_id, arrival_time, departure_time, stop_id, stop_name, stop_sequence, stop_lat, stop_lon, update_time)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE 
route_short_name = VALUES(route_short_name), 
service_id = VALUES(service_id), 
route_id = VALUES(route_id), 
direction_id = VALUES(direction_id), 
arrival_time = VALUES(arrival_time), 
departure_time = VALUES(departure_time), 
stop_id = VALUES(stop_id), 
stop_name = VALUES(stop_name), 
stop_sequence = VALUES(stop_sequence), 
stop_lat = VALUES(stop_lat), 
stop_lon = VALUES(stop_lon), 
update_time = VALUES(update_time)
""")

#Defining the cursor for deleting the buses.
delete_buses = ("DELETE FROM buses WHERE trip_id=%s")

#Converting the time because our dataset has 24:00 it should be 00:00 in order to process.
def convert_time(time_str):
    try:
        parts = time_str.split(':')
        hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
        if hours >= 24:
            hours -= 24
        return f"{hours:02}:{minutes:02}:{seconds:02}"
    except ValueError as e:
        print(f"Error converting time: {time_str} - {e}")
        return None

stop_times['arrival_time'] = stop_times['arrival_time'].apply(convert_time)
stop_times['departure_time'] = stop_times['departure_time'].apply(convert_time)
stop_times = stop_times.dropna(subset=['arrival_time', 'departure_time'])


#Reading User input for start and end time. They should be in HH:MM format.
def get_time_input(prompt):
    while True:
        try:
            time_str = input(prompt).strip()

            # If input is in HHMM format (e.g., "1000")
            if len(time_str) == 4 and time_str.isdigit():
                time_str = f"{time_str[:2]}:{time_str[2:]}:00"
                #time_str_datetime=datetime.strptime(time_str, '%H:%M:%S')

            elif len(time_str)==13 and time_str[:8].isdigit and time_str[8].isspace() and time_str[-4:].isdigit:
                date_part=f"{time_str[:4]}-{time_str[4:6]}-{time_str[6:8]}"
                time_part=f"{time_str[9:11]}:{time_str[11:13]}:00"
                time_str = f"{date_part} {time_part}"


                '''
                date_str_datetime=datetime.strptime(date_str, '%Y-%m-%d').time()
                time_str_datetime=datetime.strptime(time_str, '%H:%M:%S')
                time_str_datetime=datetime.combine(date_str_datetime, time_str_datetime)
                '''
                

            else:
                raise ValueError

            # Validate the time format
            
            return time_str
        except ValueError:
            print("Invalid time format. Please enter time in HHMM or YYYYMMDD HHMM format.")

#Finding todays service id.
def get_service_ids_for_date(date):
    day_index = date.weekday()
    days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    day_column = days[day_index]
    service_ids = calendar[calendar[day_column] == 1]['service_id'].tolist()
    return service_ids

#Finding the current buses with using the time and service id.
def get_buses_at_time(target_time_str, service_ids):
    filtered_stop_times = stop_times[
        (stop_times['arrival_time'] == target_time_str) | 
        (stop_times['departure_time'] == target_time_str)
    ]
    
    if filtered_stop_times.empty:
        return []

    
    
    filtered_stop_times = pd.merge(filtered_stop_times, trips, on='trip_id', how='left')
    filtered_stop_times = filtered_stop_times[filtered_stop_times['service_id'].isin(service_ids)]
    
    if filtered_stop_times.empty:
        return []
    merged_df = pd.merge(filtered_stop_times, routes, on='route_id', how='left')
    merged_df = pd.merge(merged_df, stops, on='stop_id', how='left')
    
    bus_data = []
    for _, row in merged_df.iterrows():
        routeNumber = row['route_short_name']
        tripID = row['trip_id']
        routeID = row['route_id']
        stopName = row['stop_name']
        arrivalTime = row['arrival_time']
        departureTime = row['departure_time']
        serviceID = row['service_id']
        stopSequence = row['stop_sequence']
        directionID = row['direction_id']
        stopID = row['stop_id']
        stopLat = row['stop_lat']
        stopLon = row['stop_lon']
        print(
        f"Route: {routeNumber}, Trip ID: {tripID}, Stop: {stopID}, Arrival Time: {arrivalTime}, Departure Time: {departureTime}, "
        f"Stop Name: {stopName}, Direction ID: {directionID}, Stop Lat: {stopLat}, Stop Lon: {stopLon}, Route ID: {routeID}, Service ID: {serviceID}, Stop Sequence: {stopSequence}")
        bus_data.append((routeNumber, tripID, serviceID, routeID, directionID, arrivalTime, departureTime,
                         stopID, stopName, stopSequence, stopLat, stopLon))   
    return bus_data

def create_stop_markers(stops_df, mymap, stop_markers):
    for stop in stops_df.itertuples():
        marker = Marker(location=[stop.stop_lat, stop.stop_lon], draggable=False)
        popup_content = HTML()
        popup_content.value = f"Stop: {stop.stop_name}, Stop Number: {stop.stop_id}"
        popup = Popup(location=[stop.stop_lat, stop.stop_lon], child=popup_content)
        marker.popup = popup
        mymap.add_layer(marker)
        stop_markers[stop.stop_id] = marker
    return stop_markers

def create_bus_marker(existing_markers, bus_data):
    stop_lat = bus_data['stop_lat']
    stop_lon = bus_data['stop_lon']
    # Offset step to prevent overlapping markers
    offset = 0.0001

    # Check if a marker already exists at the same location
    for marker in existing_markers.values():
        if abs(marker.location[0] - stop_lat) < offset and abs(marker.location[1] - stop_lon) < offset:
            stop_lat += offset
            stop_lon += offset
            offset += 0.0001

    # Create a new marker at the adjusted location
    icon = AwesomeIcon(name='bus', marker_color='red', icon_color='white')
    new_marker = Marker(icon=icon, location=[stop_lat, stop_lon], draggable=False)
    
    return new_marker

    

def update_popup(marker, bus_data):
    popup_content = HTML()
    popup_content.value = f"Route Name: {bus_data[0]}, Trip ID: {bus_data[1]}, Stop Name: {bus_data[8]}, Arrival Time: {bus_data[5]}"
    marker.popup = Popup(location=marker.location, child=popup_content)

def dynamic_update_service(start_time_str, end_time_str):

    if len(start_time_str)==19 and len(end_time_str)==19:
        start_datetime = datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S')
        end_datetime = datetime.strptime(end_time_str, '%Y-%m-%d %H:%M:%S')

    elif len(start_time_str)==8 and len(end_time_str)==8:    
        today_date = datetime.now().date()
        start_time_str = datetime.strptime(start_time_str, '%H:%M:%S').time()
        end_time_str = datetime.strptime(end_time_str, '%H:%M:%S').time()
        start_datetime = datetime.combine(today_date, start_time_str)
        end_datetime = datetime.combine(today_date, end_time_str)
        if end_time_str < start_time_str:
            end_datetime += timedelta(days=1)

    else:
        raise ValueError("Invalid time format. Please ensure start and end times are either in 'HH:MM:SS' or 'YYYY-MM-DD HH:MM:SS' format.")
    
    current_datetime = start_datetime
    markers = {}
    stop_markers = {}

    mapcenter = (stops['stop_lat'].mean(), stops['stop_lon'].mean())
    mymap = Map(center=mapcenter, zoom=12, basemap=basemaps.OpenStreetMap.Mapnik)
    
    stop_markers = create_stop_markers(stops, mymap, stop_markers)
    display(mymap)
    
    while current_datetime <= end_datetime:
        current_time_str = current_datetime.strftime('%H:%M:%S')
        print(f"\nCurrent Time: {current_time_str}")

        current_service_ids = get_service_ids_for_date(current_datetime.date())
        buses = get_buses_at_time(current_time_str, current_service_ids)
        
        if not buses:
            print("No new bus data available, retaining previous state.")

        else:
            startop= round(time.time() * 1000) 
            print("ŞU SAATTE BAŞLADIK ", startop)
            try:
            # Insert each bus data row into the database
                for data in buses:
                    cursor.execute(insert_or_update_buses,(*data,current_datetime))

            # Commit the transaction to save the data in the database
                    db.commit()

            except mysql.connector.Error as err:
                print(f"Error: {err}")
                db.rollback()  # Rollback the transaction on error

             # trip_id, stop_lat, stop_lon
            for bus_data in buses:
                trip_id = bus_data[1]
                stop_lat, stop_lon = bus_data[10], bus_data[11]
    
                if trip_id in markers:
                    # Update existing marker location and popup
                    marker = markers[trip_id]
                    if marker.location != [stop_lat, stop_lon]:  # Only update if the location has changed
                        marker.location = [stop_lat, stop_lon]
                        update_popup(marker, bus_data)
                else:
                    new_marker = create_bus_marker(markers, {
                        'stop_lat': stop_lat,
                        'stop_lon': stop_lon,})
                    
                    update_popup(new_marker, bus_data)
                    mymap.add_layer(new_marker)
                    markers[trip_id] = new_marker
                    
            try:
                bus_trip_ids = [bus[1] for bus in buses]
                for trip_id in list(markers.keys()):
                    if trip_id not in bus_trip_ids:
                        mymap.remove_layer(markers[trip_id])
                        del markers[trip_id]
                        print(f"Trip ID {trip_id} has completed its trip and has been removed.")
                        
                        cursor.execute(delete_buses, (trip_id,))
                        db.commit()
                        
            except mysql.connector.Error as err:
                                print(f"Error: {err}")
                                db.rollback()  # Rollback the transaction on error
                
            print("ROUNDTIME: ", round(time.time() * 1000)-startop)
        current_datetime += timedelta(seconds=update_interval)
        time.sleep(5)

start_time_str = get_time_input("Enter start time (YYYYMMDD HHMM or HHMM): ")
end_time_str = get_time_input("Enter end time (YYYYMMDD HHMM or HHMM): ")
dynamic_update_service(start_time_str, end_time_str)

Map(center=[41.39582465849056, 27.357291164150944], controls=(ZoomControl(options=['position', 'zoom_in_text',…


Current Time: 23:00:00
No new bus data available, retaining previous state.

Current Time: 23:30:00
No new bus data available, retaining previous state.

Current Time: 00:00:00
No new bus data available, retaining previous state.

Current Time: 00:30:00
No new bus data available, retaining previous state.

Current Time: 01:00:00
No new bus data available, retaining previous state.

Current Time: 01:30:00
No new bus data available, retaining previous state.

Current Time: 02:00:00
No new bus data available, retaining previous state.

Current Time: 02:30:00
No new bus data available, retaining previous state.

Current Time: 03:00:00
No new bus data available, retaining previous state.

Current Time: 03:30:00
No new bus data available, retaining previous state.

Current Time: 04:00:00
No new bus data available, retaining previous state.

Current Time: 04:30:00
No new bus data available, retaining previous state.

Current Time: 05:00:00
No new bus data available, retaining previous state.

KeyboardInterrupt: 